# NoteBook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [60]:

TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

In [3]:
sparse_train_data=np.loadtxt(TRAINING_DATA_FILE, delimiter=' ',dtype=int)

In [4]:
sparse_test_data=np.loadtxt(TEST_DATA_FILE, delimiter=' ',dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
print('No. of rows in training data = ', sparse_train_data.shape[0])
print('No. of rows in test data = ', sparse_test_data.shape[0])

No. of rows in training data =  258333
No. of rows in test data =  117618


In [7]:
print('No. of emails in training data = ', np.unique(sparse_train_data[:,0]).size)

No. of emails in training data =  4015


In [8]:
print('No. of emails in test data = ', np.unique(sparse_test_data[:,0]).size)

No. of emails in test data =  1724


### How to create an empty DataFrame

In [9]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names=np.unique(sparse_train_data[:,0])

In [12]:
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
sparse_train_data.shape[0]

258333

In [14]:
sparse_train_data[1][3]

2

In [15]:
full_train_data= pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True)

In [16]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,
                     freq_idx=3):
    
    
    
    column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
    doc_id_names=np.unique(sparse_matrix[:,0])
    full_matrix= pd.DataFrame(index=index_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr= sparse_matrix[i][doc_idx]
        word_id= sparse_matrix[i][word_idx]
        label= sparse_matrix[i][cat_idx]
        occurance=sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID']= int(doc_nr)
        full_matrix.at[doc_nr,'CATEGORY']= label
        full_matrix.at[doc_nr,word_id]= occurance
        
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [17]:
%%time

full_train_data=make_full_matrix(sparse_train_data,VOCAB_SIZE)

Wall time: 24.5 s


In [18]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating the probability of SPAM

In [19]:
full_train_data.CATEGORY.size

4015

In [20]:
full_train_data.CATEGORY.sum()

1250

In [21]:
probability_spam= full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size

In [22]:
print('Probability of Spam data = ', probability_spam)

Probability of Spam data =  0.31133250311332505


## Total Number of Words/ Tokens

In [23]:
full_train_features =  full_train_data.loc[:, full_train_data.columns!='CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
email_lengths= full_train_features.sum(axis=1)
email_lengths.size

4015

In [25]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [26]:
total_word_count= email_lengths.sum()

In [27]:
total_word_count

430983

## Number of Tokens in Spam nd Ham Emails

In [28]:
spam_lengths=email_lengths[full_train_data.CATEGORY==1]

In [29]:
spam_lengths.shape

(1250,)

In [30]:
spam_lengths.head()

DOC_ID
0     87
1     53
2     40
3    181
4     43
dtype: int64

In [31]:
spam_lengths.tail()

DOC_ID
1885     45
1887     22
1889    115
1890     26
1895    162
dtype: int64

In [32]:
spam_word_count=spam_lengths.sum()
spam_word_count

178671

In [33]:
ham_lengths=email_lengths[full_train_data.CATEGORY==0]

In [34]:
ham_lengths.shape

(2765,)

In [35]:
ham_word_count=ham_lengths.sum()
ham_word_count

252312

In [36]:
total_word_count-spam_word_count-ham_word_count

0

## Summing the Tokens Occuring in Spam

In [37]:
full_train_features.shape

(4015, 2500)

In [38]:
train_spam_tokens= full_train_features.loc[full_train_data.CATEGORY==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
train_spam_tokens.shape

(1250, 2500)

In [40]:
summed_spam_tokens = train_spam_tokens.sum(axis=0)+1

In [41]:
summed_spam_tokens

0       2177
1        936
2       1220
3       2026
4       1211
        ... 
2495      25
2496       1
2497      23
2498       3
2499      23
Length: 2500, dtype: int64

In [42]:
train_ham_tokens= full_train_features.loc[full_train_data.CATEGORY==0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
summed_ham_tokens=train_ham_tokens.sum(axis=0)+1

In [55]:
prob_tokens_spam = summed_spam_tokens / (spam_word_count + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012016
1    0.005166
2    0.006734
3    0.011183
4    0.006684
dtype: float64

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [56]:
prob_tokens_nonspam = summed_ham_tokens / (ham_word_count + VOCAB_SIZE)
#prob_tokens_nonspam.sum()

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [57]:
prob_tokens_all = full_train_features.sum(axis=0) / total_word_count

# P(Token) - Probability that Token Occurs 

In [58]:
prob_tokens_all

0       0.017769
1       0.008172
2       0.007569
3       0.006873
4       0.006543
          ...   
2495    0.000081
2496    0.000086
2497    0.000077
2498    0.000074
2499    0.000051
Length: 2500, dtype: float64

In [59]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

In [49]:
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

In [50]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [51]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)

In [61]:
X_test.shape

(5739, 2500)

In [62]:
sparse_test_data.shape

(117618, 4)